# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
weather_df = pd.read_csv("../output_data/weather_df.csv")
weather_df


,City,Lat,Lon,Maximum Temp,Humidity,Cloudiness,Wind,Country,Date
0,vostok,46.4856,135.8833,50.63,82,99,3.49,RU,1623012017
1,rawson,-43.3002,-65.1023,51.62,46,31,17.67,AR,1623011946
2,mar del plata,-38.0023,-57.5575,55.09,66,0,1.01,AR,1623012018
3,gualeguaychu,-33.0094,-58.5172,50.65,73,7,7.52,AR,1623012018
4,cockburn town,21.4612,-71.1419,80.06,80,7,20.74,TC,1623012019
...,...,...,...,...,...,...,...,...,...
569,voh,-20.9667,164.7000,68.68,61,95,4.88,NC,1623012240
570,mount isa,-20.7333,139.5000,42.57,70,0,0.00,AU,1623012240
571,borogontsy,62.6683,131.1683,53.64,57,100,6.91,RU,1623012240
572,stromness,58.9650,-3.2960,52.05,77,92,0.76,GB,1623012241


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
gmaps.configure(api_key=g_key)

coordinates = weather_df[["Lat", "Lon"]].astype(float)
humidity = weather_df["Humidity"].astype(float)

In [9]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
# Add a "Hotel Name" column to the DataFrame.
weather_df["Hotel Name"] = ""


hotel_df = weather_df.loc[(
    weather_df["Maximum Temp"] > 60)]
hotel_df = hotel_df.loc[(
    hotel_df["Maximum Temp"] < 75)]
hotel_df = hotel_df.loc[(
    hotel_df["Humidity"] < 75)]

#reduce dataframe to select columns
hotel_df = hotel_df[["City", "Country", "Lat", "Lon", "Hotel Name"]]
hotel_df = hotel_df.dropna(how='any')
hotel_df

,City,Country,Lat,Lon,Hotel Name
6,cherskiy,RU,68.7500,161.3000,
43,hami,CN,42.8000,93.4500,
59,korla,CN,41.7597,86.1469,
62,kelowna,CA,49.8831,-119.4857,
67,fairbanks,US,64.8378,-147.7164,
...,...,...,...,...,...
516,mountain home,US,43.1330,-115.6912,
555,bilibino,RU,68.0546,166.4372,
565,skoghall,SE,59.3232,13.4655,
566,guerrero negro,MX,27.9769,-114.0611,


In [15]:
#Create parameters for gmaps iteration 
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    # get lat and lng from df
    lat = row["Lat"]
    lng = row["Lon"]

    # change location based on each lat/lng from dataframe 
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #define city variable
    city = row["City"]
    
    # create url and make API request
    print(f"Results for {'city'}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    #cities that have a hotel meeting the parameters defined will print the closest hotel
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    #cities that do not have a hotel meeting the parameters will indicate this and skip.
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Results for city: cherskiy.
Missing field/result... skipping.
------------
Results for city: hami.
Missing field/result... skipping.
------------
Results for city: korla.
Missing field/result... skipping.
------------
Results for city: kelowna.
Missing field/result... skipping.
------------
Results for city: fairbanks.
Missing field/result... skipping.
------------
Results for city: east london.
Missing field/result... skipping.
------------
Results for city: hailey.
Missing field/result... skipping.
------------
Results for city: castro.
Missing field/result... skipping.
------------
Results for city: cape town.
Missing field/result... skipping.
------------
Results for city: jinchang.
Missing field/result... skipping.
------------
Results for city: diego de almagro.
Missing field/result... skipping.
------------
Results for city: college.
Closest hotel is Pike's Waterfront Lodge.
------------
Results for city: hermanus.
Closest hotel is Misty Waves Boutique Hotel.
------------
Result

In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel.iterrows()]
locations = hotel[["Lat", "Lon"]]

In [19]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

#Create gmap and add heat layer.
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))